In [ ]:
#Корпус я составил на основе данных из НКРЯ, большую часть предложений взял из подкурпуса социальных сетей.
#Там есть сленг, например, слова "имба", "ботать", "кринж" и т.п;
#Некоторые общепринятые сокращения вроде "г.", "м.", "др.";
#Слова с дефисным написанием, например, "мать-и-мачеха","школа-гимназия" и т.п.;
#Слова, в которых для эмоциональности удвоены буквы: "Ужжжасно","Ур-ра-а";
#Имена собственные
#Аббревиатуры

#Для основы разметки я взял теггинг Spacy с некоторыми изменениями: предлог PREP, все союзы CONJ. 
#Он мне показался наиболее удобным для разметки, так как есть отдельные теги для имен собственных, для символов, пунктуации и т.п.

In [2]:
pip install stanza

  Obtaining dependency information for stanza from https://files.pythonhosted.org/packages/88/4f/064015f46172c860b02148db65acd67e4925900b426f66cd0f5729d1c0d1/stanza-1.6.1-py3-none-any.whl.metadata
  Obtaining dependency information for emoji from https://files.pythonhosted.org/packages/96/c6/0114b2040a96561fd1b44c75df749bbd3c898bf8047fb5ce8d7590d2dee6/emoji-2.8.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for protobuf>=3.15.0 from https://files.pythonhosted.org/packages/c2/59/f89c04923d68595d359f4cd7adbbdf5e5d791257945f8873d88b2fd1f979/protobuf-4.24.4-cp310-abi3-win_amd64.whl.metadata
   ---------------------------------------- 881.2/881.2 kB 3.1 MB/s eta 0:00:00
   --------------------------------------- 430.5/430.5 kB 13.6 MB/s eta 0:00:00
   ---------------------------------------- 358.9/358.9 kB 7.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [32]:
import stanza
import json
import jsonlines

stz = stanza.Pipeline(lang='ru', processors='tokenize,pos')

2023-10-09 15:53:56 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-10-09 15:53:57 INFO: Loading these models for language: ru (Russian):
| Processor | Package          |
--------------------------------
| tokenize  | syntagrus        |
| pos       | syntagrus_charlm |

2023-10-09 15:53:57 INFO: Using device: cpu
2023-10-09 15:53:57 INFO: Loading: tokenize
2023-10-09 15:53:57 INFO: Loading: pos
2023-10-09 15:53:58 INFO: Done loading processors!


In [33]:
with open("корпус.txt", 'r', encoding='utf-8') as textt:
    for line in textt:
        doc = stz(line)
        for snt in doc.sentences:
            for word in snt.words:
                dictty={"word": word.text, "pos": word.upos}
                with jsonlines.open('razbor_stz.jsonl', mode='a') as razbor:
                    razbor.write(dictty)

In [96]:
#происходит соотношение моего теггинга и станза


with jsonlines.open('razbor_stz.jsonl', mode='r') as stz:
    stz_ar=[]
    for line in stz:
        if line not in stz_ar:
            stz_ar.append(line)
        

with jsonlines.open('razbor_0.jsonl', mode='r') as f0:
    _0_ar=[]
    for line in f0:
        if line not in _0_ar:
            _0_ar.append(line)
        
for i in stz_ar:
    if i['pos']=='CCONJ' or i['pos']=='SCONJ':
        i['pos']='CONJ'
    elif i['pos']=='AUX':
        i['pos']='VERB'
    elif i['pos']=='ADP':
        i['pos']='PREP'
    elif i['pos']=='DET':
        i['pos']='ADJ'

In [155]:
#происходит высчитывание количества правильных определений, по отношению к эталону
#затем определяю точность, делю количество праавильных определений на общее кол-во (кроме пунктуации)

match=0
num=len(_0_ar)


for i in _0_ar:
        for a in stz_ar:
            if i['word'] == a['word']:
                if i['pos'] == a['pos']:
                    match+=1

print('По моему корпусу точность Stanza составляет', round(match/num*100, 2), '%')
#Это результат первого теггера, stanza 

По моему корпусу точность Stanza составляет 77.51 %


In [91]:
#Делаем то же самое с pymorphy
from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()
from pymorphy2.tokenizers import simple_word_tokenize



with open("корпус.txt", 'r', encoding='utf-8') as texttt:
    for line in texttt:
        for token in simple_word_tokenize(line):
            doc = m.parse(token)[0]
            dictty={"word": doc.word, "pos": doc.tag.POS}
            with jsonlines.open('razbor_pmr.jsonl', mode='a') as razbor:
                razbor.write(dictty)

In [152]:
pmr_arr=[]
with jsonlines.open('razbor_pmr.jsonl', mode='r') as pmr:
    for line in pmr:
        if line not in pmr_arr:
            pmr_arr.append(line)
        
for i in pmr_arr:
    if i['pos']=='ADJS' or i['pos']=='ADJF':
        i['pos']='ADJ'
    elif i['pos']=='PRTS' or i['pos']=='INFN':
        i['pos']='VERB'
    elif i['pos']=='PRCL':
        i['pos']='PART'
    elif i['pos']=='ADVB':
        i['pos']='ADV'
    elif i['pos']=='NPRO':
        i['pos']='PRON'
    elif i['pos']=='NUMR':
        i['pos']='NUM'

In [159]:
match_2=0
num=len(_0_ar)


for i in _0_ar:
        for a in pmr_arr:
            if i['word'] == a['word']:
                if i['pos'] == a['pos']:
                    match_2+=1

print('По моему корпусу точность pymorphy составляет', round(match_2/num*100, 2), '%')
#Это результат второго теггера, pymorphy 

По моему корпусу точность pymorphy составляет 78.47 %


In [114]:
#Теперь возьмем Наташу
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    
    PER,
    NamesExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [166]:
with open("корпус.txt", 'r', encoding='utf-8') as texttt:
    for line in texttt:
        doc = Doc(line)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        doc.tokens
        for i in doc.tokens:
            dictty={"word": i.text, "pos": i.pos}
            with jsonlines.open('razbor_ntsh.jsonl', mode='a') as razbor:
                razbor.write(dictty)
            

In [167]:
ntsh_arr=[]
with jsonlines.open('razbor_ntsh.jsonl', mode='r') as ntsh:
    for line in ntsh:
        if line not in ntsh_arr:
            ntsh_arr.append(line)

for i in ntsh_arr:
    if i['pos']=='CCONJ' or i['pos']=='SCONJ':
        i['pos']='CONJ'
    elif i['pos']=='AUX':
        i['pos']='VERB'
    elif i['pos']=='ADP':
        i['pos']='PREP'
    elif i['pos']=='DET':
        i['pos']='ADJ'
        
match_3=0
num=len(_0_ar)


for i in _0_ar:
        for a in ntsh_arr:
            if i['word'] == a['word']:
                if i['pos'] == a['pos']:
                    match_3+=1

print('По моему корпусу точность natasha составляет', round(match_3/num*100, 2), '%')
#Это результат третьего теггера, natasha

По моему корпусу точность natasha составляет 70.81 %


 Получилось, что лучший теггер это pymorphy
 
 Кажется, самое очевидное, что модет быть в отзыве, это конструкция типа глаг.+нар., в котором наречие чаще всего показывает отношение говорящего. 
 Также, мне кажется, показательными будут наречия меры степени типа "очень" с наречиями или прилагательными
 У меня был сериал, поэтому еще, наверное важно смотреть на конструкции типа "сериал"+прил.
 Мне кажется, эти шаблоны выделят основные слова, которые отображают отношение автора текста и, соответсвенно, тональность текста

In [202]:
#пытаюсь написать чанкер

from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()
from pymorphy2.tokenizers import simple_word_tokenize
import csv


with open("корпус.txt", 'r', encoding='utf-8') as texttt:
    for line in texttt:
        for token in simple_word_tokenize(line):
            doc = m.parse(token)[0]
            if doc.tag.POS!=None:
                with open("корпус1.txt", 'a', encoding='utf-8') as f:
                    f.write('-'.join([doc.word, doc.tag.POS]) + ' ')


Возьмем, к примеру, шаблоны типа "не + глагол", "частица + сущ." и "предл. + прил. + сущ."

In [241]:
with open("корпус1.txt", 'r', encoding='utf-8') as te:
    for line in te:
        tookens=''.join(line)
        
tokens_sp=tookens.split()

for i in range(len(tokens_sp)):
    meta1=tokens_sp[i].split('-')
    try:
        meta2=tokens_sp[i+1].split('-')
    except:
        Exception
        
    if meta1[0] == 'не':
        if meta2[1] == 'VERB' or meta2[1] == 'INFN':
            print(meta1[0] + ' ' + meta2[0])

не признать
не стоят


Он нашел всего две формы, но, кажется, других и нет. Все работает. Проверим на других шаблонах

In [249]:
with open("корпус1.txt", 'r', encoding='utf-8') as te:
    for line in te:
        tookens=''.join(line)
        
tokens_sp=tookens.split()

for i in range(len(tokens_sp)):
    meta1=tokens_sp[i].split('-')
    try:
        meta2=tokens_sp[i+1].split('-')
    except:
        Exception
        
    if meta1[1] == 'PRCL':
        if meta2[1] == 'NOUN':
            print(meta1[0] + ' ' + meta2[0])

это ученица
бы счастье
ведь роспись
это образ
бы кринж


In [245]:
with open("корпус1.txt", 'r', encoding='utf-8') as te:
    for line in te:
        tookens=''.join(line)
        
tokens_sp=tookens.split()

for i in range(len(tokens_sp)):
    meta1=tokens_sp[i].split('-')
    try:
        meta2=tokens_sp[i+1].split('-')
        meta3=tokens_sp[i+2].split('-')
    except:
        Exception
    if meta1[1] == 'PREP':
        if meta2[1] == 'ADJF' or meta1[1] == 'ADJS':
            if meta3[1] == 'NOUN':
                print(meta1[0] + ' ' + meta2[0] + ' ' + meta3[0])

на летней площадке
от нашей кухни
по воронежской области
в жилом доме
по всей площади
о своём образовании
у чужих ворот
в какие рамки
при всем неприятии
в ботанических книгах
на одном дыхании


Получается как-то так. Все работает. Прошлая домашка, как я понял, была сделала не очень правильно, поэтому не вижу смысла на ней применять эту штуку. 